In [ ]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq


In [ ]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [ ]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='Among all the questions, 30% of them have the answer A and 70% have the answer B.'
    else:
        bias='Among all the questions, 70% of them have the answer A and 30% have the answer B.'
    template="""Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
[ Does the clause contain a license granted by one party to its counterparty?
%s

Clause: %s ]

Rewrite Prompt:
"""%(bias,fact)
    return template

In [ ]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    ans2=get_output(ans1+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("Self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(now['ans']),'prompt':ans1}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

